# Тема “POS-tagger и NER”

## Задание 1. Написать теггер на данных с русским языком
### 1.	проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации


In [ ]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu

--2022-10-21 18:03:47--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736565 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train-a.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   196MB/s    in 0.2s    

2022-10-21 18:03:48 (196 MB/s) - ‘ru_syntagrus-ud-train-a.conllu’ saved [40736565/40736565]

--2022-10-21 18:03:48--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185

In [ ]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pyconll

In [ ]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train-a.conllu')
full_train_b = pyconll.load_from_file('ru_syntagrus-ud-train-b.conllu')
full_train_c = pyconll.load_from_file('ru_syntagrus-ud-train-c.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-test.conllu')

full_train.extend([*full_train_b, *full_train_c])

In [ ]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])

fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])

fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])
#fdata_sent_test = fdata_sent_test[0]

In [8]:
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger
from nltk.tag import SequentialBackoffTagger

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
default_tagger = DefaultTagger('NN')
display(default_tagger.tag(fdata_sent_test), default_tagger.accuracy(fdata_test))

[('В', 'NN'),
 ('советский', 'NN'),
 ('период', 'NN'),
 ('времени', 'NN'),
 ('число', 'NN'),
 ('ИТ', 'NN'),
 ('-', 'NN'),
 ('специалистов', 'NN'),
 ('в', 'NN'),
 ('Армении', 'NN'),
 ('составляло', 'NN'),
 ('около', 'NN'),
 ('десяти', 'NN'),
 ('тысяч', 'NN'),
 ('.', 'NN')]

0.0

In [ ]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(fdata_sent_test), unigram_tagger.accuracy(fdata_test))

[('В', 'ADP'),
 ('советский', 'ADJ'),
 ('период', 'NOUN'),
 ('времени', 'NOUN'),
 ('число', 'NOUN'),
 ('ИТ', 'PROPN'),
 ('-', 'PUNCT'),
 ('специалистов', 'NOUN'),
 ('в', 'ADP'),
 ('Армении', 'PROPN'),
 ('составляло', 'VERB'),
 ('около', 'ADP'),
 ('десяти', 'NUM'),
 ('тысяч', 'NOUN'),
 ('.', 'PUNCT')]

0.8880238497616922

In [ ]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(fdata_sent_test), bigram_tagger.accuracy(fdata_test))

[('В', 'ADP'),
 ('советский', 'ADJ'),
 ('период', 'NOUN'),
 ('времени', 'NOUN'),
 ('число', 'NOUN'),
 ('ИТ', 'PROPN'),
 ('-', 'PUNCT'),
 ('специалистов', 'NOUN'),
 ('в', 'ADP'),
 ('Армении', 'PROPN'),
 ('составляло', 'VERB'),
 ('около', 'ADP'),
 ('десяти', 'NUM'),
 ('тысяч', 'NOUN'),
 ('.', 'PUNCT')]

0.89297989100507

In [ ]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.tag(fdata_sent_test), trigram_tagger.accuracy(fdata_test))

[('В', 'ADP'),
 ('советский', 'ADJ'),
 ('период', 'NOUN'),
 ('времени', 'NOUN'),
 ('число', 'NOUN'),
 ('ИТ', 'NOUN'),
 ('-', 'PUNCT'),
 ('специалистов', 'NOUN'),
 ('в', 'ADP'),
 ('Армении', 'PROPN'),
 ('составляло', 'VERB'),
 ('около', 'ADP'),
 ('десяти', 'NUM'),
 ('тысяч', 'NOUN'),
 ('.', 'PUNCT')]

0.8918848780611308

In [ ]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.accuracy(fdata_test) 

0.8914861161220085

### 2.	написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

In [ ]:
class NamesTagger(SequentialBackoffTagger):
    def __init__(self, model, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.model = model
            
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        return self.model.tag([word])[0][1]
            
nt = NamesTagger(trigram_tagger)
print(nt.tag(['составляло', 'период'])) 
print(nt.tag(['Армении'])) 
print(nt.tag(['.']))

[('составляло', 'VERB'), ('период', 'NOUN')]
[('Армении', 'PROPN')]
[('.', 'PUNCT')]


In [ ]:
nt.accuracy(fdata_test)

0.8845995607289115

### 3.	сравнить все реализованные методы сделать выводы:
- Все используемые методы показали не плохие результаты, и этого результата мы смогли добиться по причине очень качественных исходных данных. 

## Задание 2. Проверить насколько хорошо работает NER данные брать из http://www.labinform.ru/pub/named_entities/
### 1.	проверить NER из nltk/spacy/deeppavlov.
### 2.	написать свой NER, попробовать разные подходы.

a.	передаём в сетку токен и его соседей.

b.	передаём в сетку только токен.
            
c.	свой вариант.

#### 3.	сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [1]:
import nltk
import requests
from bs4 import BeautifulSoup
import re

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [2]:
def url_to_string(url):
    res = requests.get(url)
    #html = res.text
    soup = BeautifulSoup(res.content, 'html.parser')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [3]:
document = url_to_string('http://www.labinform.ru/pub/named_entities/')

#### ***nltk***

In [14]:
nltk.pos_tag(nltk.word_tokenize(document))[:15]

[('Размеченные', 'JJ'),
 ('коллекции', 'NNP'),
 ('для', 'NNP'),
 ('извлечения', 'NNP'),
 ('именованных', 'NNP'),
 ('сущностей', 'NNP'),
 ('LABINFORM.RU', 'NNP'),
 ('РАЗМЕЧЕННЫЕ', 'NNP'),
 ('КОЛЛЕКЦИИ', 'NNP'),
 ('ДЛЯ', 'NNP'),
 ('ИЗВЛЕЧЕНИЯ', 'NNP'),
 ('ИМЕНОВАННЫХ', 'NNP'),
 ('СУЩНОСТЕЙ', 'NNP'),
 ('Коллекция', 'NNP'),
 ('Named_Entities_5', 'NNP')]

In [20]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('DOI', 'ORGANIZATION'),
 ('Geopolit', 'ORGANIZATION'),
 ('ISMW', 'ORGANIZATION'),
 ('Intelligence', 'GPE'),
 ('Loc', 'ORGANIZATION'),
 ('Loc Имена', 'PERSON'),
 ('Loukachevitch N.', 'PERSON'),
 ('MEDIA', 'ORGANIZATION'),
 ('Org', 'ORGANIZATION'),
 ('Org Москвы', 'PERSON'),
 ('Org Правительства', 'PERSON'),
 ('Org Разметка', 'PERSON'),
 ('PDF', 'ORGANIZATION'),
 ('Russian', 'GPE'),
 ('Social Media', 'ORGANIZATION'),
 ('Библиотека', 'ORGANIZATION'),
 ('Вложенные', 'PERSON'),
 ('Коллекция', 'PERSON'),
 ('Ленина', 'PERSON'),
 ('Лукашевич', 'PERSON'),
 ('Можарова В.А.', 'PERSON'),
 ('Москва', 'PERSON'),
 ('Типы', 'PERSON'),
 ('сущностейКоличество Pers10623', 'ORGANIZATION')}

#### ***spacy***

In [1]:
!pip install -U spacy
!python -m spacy info

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 3.8 MB/s 
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.1
    Uninstalling spacy-3.4.1:
      Successfully uninstalled spacy-3.4.1
2022-10-22 10:43:37.995357: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

============================== Info about spaCy ==============================

spaCy version    3.4.2                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.10.133+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.15                        
Pipelines        en_core_web_sm (3.4.1)        



In [18]:
import spacy
from spacy import displacy
import en_core_web_sm

nlp = en_core_web_sm.load()
article = nlp(document)
displacy.render(article, jupyter=True, style='ent')

#### ***Deeppavlov***

In [4]:
! pip install deeppavlov
! python -m deeppavlov install ner_rus
! python -m deeppavlov interact -d ner_rus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 878 kB 4.4 MB/s 
     |████████████████████████████████| 6.7 MB 20.2 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 57 kB 5.3 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 26.1 MB 1.2 MB/s 
     |████████████████████████████████| 1.5 MB 42.2 MB/s 
     |████████████████████████████████| 859 kB 64.0 MB/s 
     |████████████████████████████████| 510 kB 52.2 MB/s 
     |████████████████████████████████| 55 kB 3.3 MB/s 
     |████████████████████████████████| 7.3 MB 30.4 MB/s 
     |████████████████████████████████| 8.2 MB 26.6 MB/s 
     |████████████████████████████████| 40 kB 19 kB/s 
     |████████████████████████████████| 3.8 MB 40.1 MB/s 
     |████████████████████████████████| 654 k

In [6]:
import deeppavlov
from deeppavlov import configs, build_model
deeppavlov_ner = build_model(configs.ner.ner_rus, download=False)
deeppavlov_ner([document])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2022-10-22 12:51:02.768 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_rus/word.dict]
INFO:deeppavlov.core.data.simple_vocab:[loading vocabulary from /root/.deeppavlov/models/ner_rus/word.dict]
2022-10-22 12:51:02.905 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_rus/tag.dict]
INFO:deeppavlov.core.data.simple_vocab:[loading v

NotImplementedError: ignored

так и не победил в Colab разницу версий TensorFlow